# **1) Import the Modules**

Modules are code libraries that contain a set of ready-to-use functions.

* The `ee` module allows developers to interact with Google Earth Engine using the Python programming language.
* The `geemap` module enables users to analyze and visualize GEE datasets interactively within a Jupyter environment.
* The `datetime` module supplies classes for manipulating dates and times.
* The `tabulate` module allows the user to display data in a table format.


In [ ]:
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00


In [ ]:
import ee
import geemap
import datetime
import tabulate

***Update the geemap package***

If you run into errors with this notebook, please uncomment the line below to update the geemap package to the latest version from GitHub. Restart the Kernel (Menu -> Kernel -> Restart) to take effect.

In [ ]:
# geemap.update_package()

# **2) Authentication Procedure**

This section includes instructions on how to setup the Google Earth Engine Python API on Colab and how to mount Google Drive on Colab. These steps should be completed on each new Colab session, Colab kernel reboot, or Colab virtual machine rollback/recycling due to inactivity.

## **2.1) GEE**

The `ee.Authenticate` function authenticates access to the Google Earth Engine servers while the `ee.Initialize` function initializes it.Upon executing the following cell the user will be prompted to grant Google Earth Engine access to their Google account.

**Note:** The Earth Engine API is installed by default in Google Colaboratory, so only import and authentication is required.

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ZDQdCDmx3brYj0yH7902ZUoid0_qOo29vUDohGLdflQ&tc=gkVQlQ-HgjU5e33dyVKPMHDQBSDWnTmA7ixkQ7fN6Vs&cc=942EXF33bzLDsZAlj_gZJR6lNgNdPkMiT16rB38P3lU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnOb-Cz5-amZM3MIsF__CsrREIj68ODb2wBfYZqEfu5FZGhU6NgP54

Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


# **3) Functions**

Map Visualization

In [ ]:
def rectangular_grid(projection):
  """
  Description:
    Displays the pixel grid associated with a given projection as box outlines.
    Each pixel is represented by a rectangular box outline.

  Parameters:
    projection (ee.Projection): The projection to visualize the grid for.

  Returns:
    An ee.Image object representing the pixel grid.
  """
  # Scale by 2 because we have 2 zero crossings when using the round function.
  cells = ee.Image.pixelCoordinates(projection.scale(2, 2))
  # Subtract rounded pixel coordinates from the original coordinates.
  grid = cells.subtract(cells.round())
  # Identify the zero crossings to obtain the box outlines of the grid.
  gridOutlines = grid.zeroCrossing()
  # Sum all the grid outlines to create a single image with the grid displayed.
  gridImage = gridOutlines.reduce("sum")
  # Apply a self-mask to remove pixels outside the grid outlines.
  return gridImage.selfMask()

Data Processing

In [ ]:
def set_sample_coordinates(sample):
  coordinates = sample.geometry().coordinates()
  sample = sample.set("longitude", coordinates[0])
  sample = sample.set("latitude", coordinates[1])

  return sample


def export_tasks_viewer(exportTasksIds, tableFormat: str = "plain"):
  """
  Description:
    Displays a table view which contains useful information about the provided export tasks.

  Notes:
    * Task_Id: The task identifier.
    * Task_State: One of READY, RUNNING, COMPLETED, FAILED, CANCELLED, UNSUBMITTED or UNKNOWN.
    * Task_Type: One of EXPORT_IMAGE, EXPORT_TILES, EXPORT_FEATURES, EXPORT_VIDEO.
    * Task_Attempt: Number of attempts.
    * Task_Description: A human-readable description of the task.
    * Queue_Time: The time that is taken while being in a queue.
    * Execution_Time: The time spent by the servers executing the task.
    * Completion_Time: SUm of queue and execution times.
    * Error_Message: Failure reason. Appears only if state is FAILED. May also include other fields.

  Arguments:
    exportTasksIdsList (list) (mandatory) A list of export task identifiers.
    tableFormat (str) (optional) The table format to use. Defaults to "plain".

  Returns:
    None, displays the export tasks table.
  """
  taskInfo = []
  tableHeaders = [
    "Task_Id", "Task_State", "Task_Type", "Task_Attempt", "Task_Description",
    "Queue_Time", "Execution_Time", "Completion_Time", "Error_Message"
  ]
  tableFormats = tabulate._table_formats.keys()

  if tableFormat not in tableFormats:
    raise ValueError(f"Invalid table format. Choose from: `{tableFormats}`.")

  # Populate taskInfo.
  for exportTaskId in exportTasksIds:

    taskState = ee.data.getTaskStatus(exportTaskId)[0]["state"]
    taskType = ee.data.getTaskStatus(exportTaskId)[0]["task_type"]
    taskDescription = ee.data.getTaskStatus(exportTaskId)[0]["description"]
    startTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["start_timestamp_ms"]/1000.0)
    updateTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["update_timestamp_ms"]/1000.0)
    creationTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["creation_timestamp_ms"]/1000.0)

    queueTime = None
    taskAttempt = None
    executionTime = None
    completionTime = None

    if taskState not in ["READY", "RUNNING"]:
      queueTime = (startTimestamp - creationTimestamp).total_seconds()
      executionTime = (updateTimestamp - startTimestamp).total_seconds()

    if taskState == "COMPLETED":
      taskAttempt = ee.data.getTaskStatus(exportTaskId)[0]["attempt"]
      completionTime = (updateTimestamp - creationTimestamp).total_seconds()

    try:
      errorMessage = ee.data.getTaskStatus(exportTaskId)[0]["error_message"]
    except KeyError:
      errorMessage = None  # This just means that the export task has not failed.

    taskInfo.append([exportTaskId, taskState, taskType, taskAttempt, taskDescription, queueTime, executionTime, completionTime, errorMessage])

  # Table display.
  table = tabulate.tabulate(taskInfo, headers=tableHeaders, tablefmt=tableFormat)
  print(table)

# **4) Parameters**

In [ ]:
# `Dynamic World Land Cover`
fromLabels = [1, 2, 3, 4, 5, 6, 7, 8]
toLabels = [1, 1, 1, 1, 1, 1, 1, 1]

dwStartDate = "2015-01-01"
dwEndDate = "2016-06-01"

# `Digital Elevation`
demProvider = "USGS"

# `Sentinel-1 GRD`
b1 = "VV"
b2 = "VH"

preIdentifier = "003747_00476D_87B6"
postIdentifier = "004447_005723_9D4B"

# Projections of interest.
projectionCRS = "EPSG:4326"
innerScale = 10
outerScale = 30

#
preCoverValue = 0
postCoverValue = 0
groupIdentifier = "emsr117"

dct = {
  "seed": 0,
  "tileScale": 16,
  "dropNulls": True,
  "geometries": True,
  "classBand": "class",
  "numPoints": 10,
  "classValues": [0],
  "classPoints": [25000]
}

# GEE assets.
rastersFolder = "users/stamlazaros/hua/t-h-e-s-i-s/assets/features/rasters/"
vectorsFolder = "users/stamlazaros/hua/t-h-e-s-i-s/assets/features/vectors/"
destinationPath = "users/stamlazaros/hua/t-h-e-s-i-s/assets/samples/emsr117/non_water"

DESIRED_REGIONS = [
  ee.FeatureCollection("FAO/GAUL/2015/level2")
]

UNDESIRED_REGIONS = [
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/assets/filters/emsr117/flood_surfaces"),
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/assets/filters/emsr117/water_surfaces"),
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/vectors/magnesia/area_of_interest")
]

# **5) Configuration**

In [ ]:
# `Digital Elevation Model`
demConfigs = {
  "CGIAR": {    # `SRTM Digital Elevation Data Version 4`
    "name": "CGIAR/SRTM90_V4"
  },
  "NASA": {     # `NASA NASADEM Digital Elevation`
    "name": "NASA/NASADEM_HGT/001"
  },
  "USGS": {     # `NASA SRTM Digital Elevation`
    "name": "USGS/SRTMGL1_003"
  },
  "ASTER": {      # `AG100: ASTER Global Emissivity Dataset 100-meter V003`
    "name": "NASA/ASTER_GED/AG100_003"
  }
}

# `Sentinel-1 GRD`
sarBluishVisualization = {
  "min":[-18, -25, 1],
  "max":[0, -5, 12],
  "bands": ["VV", "VH", "VVVHD"]
}

sarReddishVisualization = {
  "min": [-25, -25, 0],
  "max": [0, 0, 2],
  "bands": ["VV", "VH", "VVVHQ"]
}

bandCombinations = {
  "sum": {
    "expression": "b1 + b2",
    "name": "b1b2S"
  },
  "difference": {
    "expression": "b1 - b2",
    "name": "b1b2D"
  },
  "product": {
    "expression": "b1 * b2",
    "name": "b1b2P"
  },
  "quotient": {
    "expression": "b1 / b2",
    "name": "b1b2Q"
  },
  "ndpi": {
    "expression": "(b1 - b2) / (b1 + b2)",
    "name": "NDPI"
  }
}

# `Dynamic World LULC v1`
dwConfig = {
  "name": "GOOGLE/DYNAMICWORLD/V1"
}

dwLabelsPalette = {
  "water": "419BDF",
  "trees": "397D49",
  "grass": "88B053",
  "flooded vegetation": "7A87C6",
  "crops": "E49635",
  "shrub and scrub": "DFC35A",
  "built": "C4281B",
  "bare": "A59B8F",
  "snow and ice": "B39FE1"
}

dwVisualization = {
  "min": 0,
  "max": 8,
  "bands": ["label"],
  "palette": list(dwLabelsPalette.values())

}

# `OTSU` algorithm
coverVisualization = {
  "min": 0,
  "max": 1,
  "bands": ["OTSU"],
  "palette": ["deb887", "45b6fe"]
}

# GEE assets.
footprints = {
  "pre_event": vectorsFolder + preIdentifier,
  "post_event": vectorsFolder + postIdentifier
}

features = {
  "pre_event": rastersFolder + preIdentifier,
  "post_event": rastersFolder + postIdentifier
}

demConfig = demConfigs[demProvider]

In [ ]:
exportTasks = []

# **6) Data Processing**

Define the projections of interest.

In [ ]:
internalProjection = ee.Projection(projectionCRS).atScale(innerScale)
externalProjection = ee.Projection(projectionCRS).atScale(outerScale)

Handle the geometries.

In [ ]:
preEventFootprint = ee.FeatureCollection(footprints["pre_event"]).geometry()
postEventFootprint = ee.FeatureCollection(footprints["post_event"]).geometry()
footprint = preEventFootprint.intersection(postEventFootprint).buffer(-10000)

dict.region = footprint

DESIRED_REGIONS = ee.FeatureCollection(DESIRED_REGIONS).flatten()
UNDESIRED_REGIONS = ee.FeatureCollection(UNDESIRED_REGIONS).flatten()

desiredRegions = ee.Image.constant(1)     \
  .clipToCollection(DESIRED_REGIONS)      \
  .clip(footprint)                        \
  .int()

undesiredRegions = ee.Image.constant(0)   \
  .clipToCollection(UNDESIRED_REGIONS)    \
  .clip(footprint)                        \
  .int()

regionsMosaic = ee.ImageCollection([desiredRegions, undesiredRegions])  \
  .mosaic()                                                             \
  .selfMask()                                                           \
  .reproject(internalProjection)

Load, filter and process the rasters.

In [ ]:
# `Digital Elevation`
elevation = ee.Image(demConfig["name"]).unmask().clip(footprint)
slope = ee.Terrain.slope(elevation)

# `Dynamic World Land Cover V1`
dwCollection = ee.ImageCollection(dwConfig["name"])   \
  .filterDate(dwStartDate, dwEndDate)                 \
  .filterBounds(footprint)

# Generate a mosaic.
dwMosaic = dwCollection         \
  .mode()                       \
  .select(["label"], ["dw"])    \
  .clip(footprint)              \
  .reproject(internalProjection)

dwCover = dwMosaic.remap(**{
    "from": fromLabels,
    "to": toLabels,
    "bandName": "dw"
  })                    \
  .rename("DW_COVER")   \
  .selfMask()

# `Sentinel-1 GRD`
preEventRaster = ee.Image(features["pre_event"])
postEventRaster = ee.Image(features["post_event"])

classMask = ee.ImageCollection([preEventRaster, postEventRaster]) \
  .select("OTSU").reduce(ee.Reducer.sum()).rename("class")

otsuMask = preEventRaster.select("OTSU").eq(preCoverValue)        \
  .bitwiseAnd(postEventRaster.select("OTSU").eq(postCoverValue))  \
  .rename("OTSU").selfMask()

Engineer new bands.

In [ ]:
differenceExpression = bandCombinations["difference"]["expression"]
quotientExpression = bandCombinations["quotient"]["expression"]
ndpiExpression = bandCombinations["ndpi"]["expression"]

differenceName = bandCombinations["difference"]["name"]
quotientName = bandCombinations["quotient"]["name"]

# `PRE_VVVHD`
prevvprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", b2))

# `POST_VVVHD`
postvvpostvhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", b2))

# `VVVVD`
postvvprevvDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", b1))

# `VHVHD`
postvhprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b2),
      "b2": preEventRaster.select(b2)
    }
  })  \
  .rename(differenceName.replace("b1", b2).replace("b2", b2))

# `VVVHD`
postvvprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", b2))

# `PRE_VVVHQ`
prevvprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", b2))

# `POST_VVVHQ`
postvvpostvhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", b2))

# `VVVVQ`
postvvprevvQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", b1))

# `VHVHQ`
postvhprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b2),
      "b2": preEventRaster.select(b2)
    }
  })  \
  .rename(quotientName.replace("b1", b2).replace("b2", b2))

# `VVVHQ`
postvvprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", b2))

# `PRE_NDPI`
preNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  })  \
  .rename("PRE_NDPI")

# `POST_NDPI`
postNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  })  \
  .rename("POST_NDPI")

# `NDPID`
NDPID = postNDPI.subtract(preNDPI).rename("NDPID")

# Rename S1-GRD raster bands using a regular expression.
preEventSamplesSource = preEventRaster.regexpRename("^", "PRE_", False)
postEventSamplesSource = postEventRaster.regexpRename("^", "POST_", False)

# Add bands to both pre- & post- event rasters.
preEventRaster = preEventRaster.addBands([prevvprevhDifference, prevvprevhQuotient])
postEventRaster = postEventRaster.addBands([postvvpostvhDifference, postvvpostvhQuotient])

In [ ]:
# Generate a collection of random samples.

# Construct the pixel grid as box outlines.
rectangularGrid = rectangular_grid(externalProjection).clip(footprint)

# Create an even pixel coordinates mask (in either X or Y).
outerMask = ee.Image.pixelCoordinates(externalProjection) \
  .expression("!((b('x') + 0.5) % 2 != 0 || (b('y') + 0.5) % 2 != 0)")

# Create a cells mask.
cells = ee.Image.random() \
  .rename("ID")           \
  .clip(footprint)        \
  .multiply(1000000)      \
  .int()                  \
  .reproject(externalProjection)

# Retain only relevant cells.
cells = cells.updateMask(cells.mask().gt(0)).updateMask(outerMask).reproject(externalProjection)

# Extract stratified random samples.
samplesSource = cells.addBands([
    preEventSamplesSource, postEventSamplesSource, postvvprevvDifference,
    postvhprevhDifference, postvvprevhDifference, postvvprevvQuotient,
    postvhprevhQuotient, postvvprevhQuotient, preNDPI, postNDPI,
    NDPID, dwMosaic, dwCover, otsuMask, classMask, slope, regionsMosaic
  ])

samples = samplesSource.stratifiedSample(dict)
samples = samples.map(lambda sample: sample.set("group", groupIdentifier))

# **7) Console**

In [ ]:
print("*samples*")
print("number of samples:", samples.size().getInfo())
print("indicative sample: ", samples.first().getInfo())

print("*rasters*")
print("samples-source features:", samplesSource.bandNames().getInfo())

*samples*
number of samples: 19008
indicative sample:  {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [23.772521605532752, 39.98100393995811]}, 'id': '0', 'properties': {'ID': 999407, 'NDPID': -0.005034386968817833, 'POST_NDPI': -0.22465967753314314, 'POST_VH': -25.599593310545195, 'POST_VV': -16.2072756183167, 'POST_cover': 1, 'PRE_NDPI': -0.2196252905643253, 'PRE_VH': -30.88101623912468, 'PRE_VV': -19.75915411161627, 'PRE_cover': 1, 'VHVHD': 5.2814229285794845, 'VHVHQ': 0.8289750930577152, 'VVVHD': 14.67374062080798, 'VVVHQ': 0.524829736586936, 'VVVVD': 3.5518784932995686, 'VVVVQ': 0.8202413689758387, 'class': 2, 'constant': 1, 'dw_cover': 1, 'elevation': 0, 'label': 0, 'otsu_cover': 1, 'slope': 0}}
*rasters*
samples-source features: ['PRE_VV', 'PRE_VH', 'PRE_cover', 'POST_VV', 'POST_VH', 'POST_cover', 'VVVVD', 'VHVHD', 'VVVHD', 'VVVVQ', 'VHVHQ', 'VVVHQ', 'PRE_NDPI', 'POST_NDPI', 'NDPID', 'label', 'class', 'slope', 'elevation']


# **8) Map Visualization**

In [ ]:
Map = geemap.Map()

# Generate categorical legends.

# `Dynamic-World`
Map.add_legend(title="DW", position="bottomright", legend_dict = dwLabelsPalette)

Map.centerObject(footprint);

Map.addLayer(preEventRaster, sarReddishVisualization, "rasters: S1-GRD (pre)");
Map.addLayer(preEventRaster, coverVisualization, "rasters: binary mask (pre)");

Map.addLayer(postEventRaster, sarReddishVisualization, "rasters: S1-GRD (post)");
Map.addLayer(postEventRaster, coverVisualization, "rasters: binary mask (post)");

Map.addLayer(dwMosaic, dwVisualization, "rasters: DW");

Map.addLayer(rectangularGrid, {"palette": ["DAA520"]}, "rasters: grid");
Map.addLayer(cells.randomVisualizer(), {}, "rasters: cells");
Map.addLayer(footprint, {"color": "white"}, "vectors: footprint");
Map.addLayer(samples, {}, "vectors: samples");

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# **8) Export Data**

Submit export tasks.

In [ ]:
samples = samples.select([
  "group", "class", "dw", "NDPID", "POST_NDPI", "POST_VH", "POST_VV", "PRE_NDPI",
  "PRE_VH", "PRE_VV", "VHVHD",  "VHVHQ", "VVVHD", "VVVHQ", "VVVVD", "VVVVQ"
])

identifier = "_".join([preIdentifier, postIdentifier])

geeExportTask = ee.batch.Export.table.toAsset(**{
  "collection": samples,
  "assetId": destinationPath,
  "description": "/".join([identifier + "to_asset"])
})

geeExportTask.start()
exportTasks.append(geeExportTask.id)

samples = samples.map(set_sample_coordinates)

gdExportTask = ee.batch.Export.table.toDrive(**{
  "collection": samples,
  "folder": "samples",
  "fileNamePrefix": identifier,
  "description": [identifier, "to_gd"].join("_"),
  "fileFormat": "CSV",
  "selectors": [
    "group", "longitude", "latitude", "class", "dw", "NDPID","POST_NDPI",
    "POST_VH", "POST_VV",  "PRE_NDPI", "PRE_VH", "PRE_VV", "VHVHD",
    "VHVHQ",  "VVVHD", "VVVHQ", "VVVVD", "VVVVQ",
  ]
})

gdExportTask.start()
exportTasks.append(gdExportTask.id)

Monitor export tasks.

In [ ]:
export_tasks_viewer(exportTasks)

Task_Id                   Task_State    Task_Type        Task_Attempt    Task_Description                                       Queue_Time    Execution_Time    Completion_Time    Error_Message
QSKVUGARWENXS7MUO3QRE3LK  READY         EXPORT_FEATURES                  011979_0127A6_F935_005329_006C09_C51B_20kzzz_to_asset
5SYM444AALFG6O6QAUEOYUWE  READY         EXPORT_FEATURES                  011979_0127A6_F935_005329_006C09_C51B_20kzzz_to_cloud


-End of Notebook-